## Final Project Submission

Please fill out:
* Student name: Stephan Osterburg
* Student pace: full time
* Scheduled project review date/time: 10/29/2018
* Instructor name: Rafael Carrasco 


### Import all needed libraries 

In [2]:
# Your code here - remember to use markdown cells for comments as well!
import pandas as pd
import numpy as np

from scipy import stats, linalg

from sklearn import linear_model
from sklearn import neighbors
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error

from math import log

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
# set style
sns.set_style('ticks')
# overriding font size and line width
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})

# map visualization
import folium

# don't print matching warnings
import warnings
warnings.filterwarnings('ignore') 


### Read data

In [9]:
# read data and read date correctly
data = pd.read_csv("kc_house_data.csv", parse_dates = ['date'])

# set category and order them
data['zipcode'] = data['zipcode'].astype('str')
data['grade'] = data['grade'].astype('category', ordered = False)
data['view'] = data['view'].astype('category', ordered = True)
data['waterfront'] = data['waterfront'].astype('category', ordered = True)
data['condition'] = data['condition'].astype('category', ordered = True)

# sort data by date
data = data.sort_values(by = ['date'])

### Collecting basic informations about the data set

In [10]:
data.head() 

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
7316,2202500290,2014-05-02,435000.0,4,1.0,1450,8800,1.0,0.0,0.0,...,7,1450,0.0,1954,0.0,98006,47.5746,-122.135,1260,8942
20586,7853361370,2014-05-02,555000.0,4,2.5,3310,6500,2.0,0.0,0.0,...,8,3310,0.0,2012,0.0,98065,47.5150,-121.870,2380,5000
1040,7197300105,2014-05-02,550000.0,4,2.5,1940,10500,1.0,NaN,0.0,...,7,1140,800.0,1976,0.0,98052,47.6830,-122.114,2200,10500
20543,3438501320,2014-05-02,295000.0,2,2.5,1630,1368,2.0,0.0,0.0,...,7,1280,350.0,2009,0.0,98106,47.5489,-122.363,1590,2306
9587,472000620,2014-05-02,790000.0,3,2.5,2600,4750,1.0,0.0,0.0,...,9,1700,900.0,1951,0.0,98117,47.6833,-122.400,2380,4750


In [4]:
data.shape

(21597, 21)

In [11]:
data.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,sqft_above,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15
count,2.159700e+04,2.159700e+04,21597.000000,21597.000000,21597.000000,2.159700e+04,21597.000000,21597.000000,21597.000000,17755.000000,21597.000000,21597.000000,21597.000000,21597.000000
mean,4.580474e+09,5.402966e+05,3.373200,2.115826,2080.321850,1.509941e+04,1.494096,1788.596842,1970.999676,83.636778,47.560093,-122.213982,1986.620318,12758.283512
std,2.876736e+09,3.673681e+05,0.926299,0.768984,918.106125,4.141264e+04,0.539683,827.759761,29.375234,399.946414,0.138552,0.140724,685.230472,27274.441950
min,1.000102e+06,7.800000e+04,1.000000,0.500000,370.000000,5.200000e+02,1.000000,370.000000,1900.000000,0.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123049e+09,3.220000e+05,3.000000,1.750000,1430.000000,5.040000e+03,1.000000,1190.000000,1951.000000,0.000000,47.471100,-122.328000,1490.000000,5100.000000
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,1560.000000,1975.000000,0.000000,47.571800,-122.231000,1840.000000,7620.000000
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068500e+04,2.000000,2210.000000,1997.000000,0.000000,47.678000,-122.125000,2360.000000,10083.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,9410.000000,2015.000000,2015.000000,47.777600,-121.315000,6210.000000,871200.000000


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21597 entries, 7316 to 16580
Data columns (total 21 columns):
id               21597 non-null int64
date             21597 non-null datetime64[ns]
price            21597 non-null float64
bedrooms         21597 non-null int64
bathrooms        21597 non-null float64
sqft_living      21597 non-null int64
sqft_lot         21597 non-null int64
floors           21597 non-null float64
waterfront       19221 non-null category
view             21534 non-null category
condition        21597 non-null category
grade            21597 non-null category
sqft_above       21597 non-null int64
sqft_basement    21597 non-null object
yr_built         21597 non-null int64
yr_renovated     17755 non-null float64
zipcode          21597 non-null object
lat              21597 non-null float64
long             21597 non-null float64
sqft_living15    21597 non-null int64
sqft_lot15       21597 non-null int64
dtypes: category(4), datetime64[ns](1), float64(6), int

___

##### Initial Qbservation 
+ zipcode seems to be a string of sorts.
+ yr_renovated = float
+ sqft_basement = object
+ floors = float (do we actually have 0.5 floors?)
+ why is there a difference in sqft's?
    + sqft_above + sqft_basement != sqft_living15
    + so what are the differences in the sqft listed?
    + does it matter?
    + or what can be used for our model later?


___

In [8]:
data.isna().sum()

id                  0
date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2376
view               63
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3842
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
dtype: int64

##### Questions? 
+ Can we neglect the amount of NaN's in *waterfront* and *yr_renovated*? 
+ How about the amount of NaN under *view*?